# LSTM Model Type 1

In [1]:
import pandas as pd
import numpy as np
from numpy import array
from collections import defaultdict

import keras_tuner
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential
from itertools import chain
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import Tensor

from LSTM_Model_Type1 import split_sequence, new_split_sequence, split_position, LSTM_FitPredict, LSTM_FitPredict_Combined

### Read in Raw Data & Preprocessing

In [2]:
# Read in the raw age-sex cohort data without the remainder area
# Prepare the unique sa3 names, codes for LSTM fitting selection
raw_data = pd.read_csv('../Data/true_1000_fulldata.csv')
sa3_num = len(raw_data['SA3 Code'].unique())
sa3_names = raw_data['SA3 Name'].unique()
sa3_codes = raw_data['SA3 Code'].unique()

### Preprocessing by Splitting Population into Sex-Group

In [3]:
# Split the data by sex
age_groups = ['0-4','5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39','40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74','75-79', '80-84', '85+']
population_m_dict = defaultdict(dict)
population_f_dict = defaultdict(dict)
population_dict = defaultdict(dict)
# Save population for each year into lists
for sa3_code in sa3_codes:
    population_m_dict[sa3_code] = dict()
    population_f_dict[sa3_code] = dict()
    population_dict[sa3_code] = dict()
    for year in range(1991,2012):
        if(raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)]['Total'].size>0):
            population_m_dict[sa3_code][year] = raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)][['m0-4',
                                  'm5-9', 'm10-14', 'm15-19', 'm20-24', 'm25-29', 'm30-34', 'm35-39',
                                  'm40-44', 'm45-49', 'm50-54', 'm55-59', 'm60-64', 'm65-69', 'm70-74',
                                  'm75-79', 'm80-84', 'm85+']].values.tolist()[0]
            population_f_dict[sa3_code][year] = raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)][['f0-4', 'f5-9', 'f10-14', 'f15-19',
                                  'f20-24', 'f25-29', 'f30-34', 'f35-39', 'f40-44', 'f45-49', 'f50-54',
                                  'f55-59', 'f60-64', 'f65-69', 'f70-74', 'f75-79', 'f80-84', 'f85+']].values.tolist()[0]
            population_dict[sa3_code][year] = population_m_dict[sa3_code][year] + population_f_dict[sa3_code][year] 







### Create Dataframe for Storing Prediction Result

In [4]:
output = pd.DataFrame(index = range(sa3_num*36), columns = ['Code','Area name','Sex','Age group',2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011])
for sa3_code in range(0,sa3_num):
    for age_group in range(18):
        output.loc[sa3_code*36+age_group] = {'Code':sa3_codes[sa3_code],'Area name':sa3_names[sa3_code],'Sex':'Females','Age group':age_groups[age_group]}
        output.loc[sa3_code*36+18+age_group] = {'Code':sa3_codes[sa3_code],'Area name':sa3_names[sa3_code],'Sex':'Males','Age group':age_groups[age_group]}

In [5]:
output

,Code,Area name,Sex,Age group,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,10101,Goulburn - Yass,Females,0-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10101,Goulburn - Yass,Females,5-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10101,Goulburn - Yass,Females,10-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10101,Goulburn - Yass,Females,15-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10101,Goulburn - Yass,Females,20-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,80109,Woden,Males,65-69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11696,80109,Woden,Males,70-74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11697,80109,Woden,Males,75-79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11698,80109,Woden,Males,80-84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Start LSTM Fitting & Predicting

##### Define Variable for Splitting the Full Dataset

In [6]:
# choose a number of time steps
n_steps = 1

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 18
n_age_groups = 18

# Obtain the split position for the training set, validation set, and the first input_x
train_bounds, val_bounds, test_bounds = split_position(n_steps, train_start = 1991, train_end = 2001, fixed_num_val = 2)

In [7]:
def make_dataset(df, input_width, label_width, shift):
    def create_window(tensor):
        #input -> length of time series used for training
        #shift -> how far off prediction is from last input
        #label -> points to predict
        total_window_size = input_width + shift
        label_start = total_window_size - label_width

        input_bounds = slice(0, input_width)
        label_bounds = slice(label_start, None)

        inputs = tensor[:,input_bounds,:]
        labels = tensor[:,label_bounds,:]

        inputs.set_shape([None, input_width, None])
        labels.set_shape([None, label_width, None])

        return inputs, labels
    
    total_window_size = input_width + shift
    
    arr = np.array(df, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
      data=arr,
      targets=None,
      sequence_length=total_window_size,
      sequence_stride=1,
      shuffle=False,
      batch_size=32,)
    
    ds = ds.map(create_window)
    
    return ds

##### LSTM Model Defining

In [8]:
'''Function for Defining the LSTM Model'''
def lstm_model(lstm1_units, activ, optimizer, loss_fun):
    model = Sequential()
    model.add(LSTM(lstm1_units, activation=activ, input_shape=(n_steps, n_features)))
    model.add(Dense(n_age_groups))
    model.compile(optimizer=optimizer, loss=loss_fun, metrics=["mean_absolute_percentage_error"])
    return model

##### LSTM Model Tunning

In [9]:
# Define HYPERPARAMETERS
lstm1_min = 100
lstm1_max = 1000
lstm1_step = 100
activ_functions = ["relu"]
optimizers = ["adam","adagrad"]
loss_functions = ["mse"]

'''In-Build Function for LSTM Model Tuning'''
def build_model(hp):
    lstm1_units = hp.Int("LSTM units", min_value=lstm1_min,max_value=lstm1_max,step=lstm1_step)
    activation = hp.Choice("Activation Function", activ_functions)
    optimizer = hp.Choice("Optimizer", optimizers)
    loss_fun = hp.Choice("Loss Function", loss_functions)
    model = lstm_model(lstm1_units, activation, optimizer, loss_fun)
    return model

# Generate the tuned model with keras_tuner package
tuner = keras_tuner.RandomSearch(hypermodel=build_model, objective=keras_tuner.Objective("val_mean_absolute_percentage_error", "min"))

Metal device set to: Apple M1 Pro


2022-08-23 02:32:37.875299: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-23 02:32:37.875415: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


##### Generate Prediction Result

Projection for combined sex label

In [10]:
# # Generate Prediction Result of each age-cohort for all areas and store them into csv
# output = LSTM_FitPredict_Combined(sa3_codes, population_dict, n_steps, train_bounds, val_bounds, test_bounds, n_features, 1000, 2002, tuner, output)
# output.to_csv('Iterative_predict_output_Step1_1000epoch_combined_tftensor_nonneg.csv',index=False,header=True)

In [11]:
# Generate each sex's Prediction Result of each age-cohort for all areas and store them into csv
output = LSTM_FitPredict(sa3_codes, population_m_dict, n_steps, train_bounds, val_bounds, test_bounds, n_features, 1000, "Males", 2002, tuner, output)
output = LSTM_FitPredict(sa3_codes, population_f_dict, n_steps, train_bounds, val_bounds, test_bounds, n_features, 1000, "Females", 2002, tuner, output)
output.to_csv('Iterative_predict_output_Step1_1000_tftensor_nonneg.csv',index=False,header=True)

2022-08-23 02:32:38.166334: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-23 02:32:38.622386: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 02:32:39.194339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2022-08-23 02:33:24.894528: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 02:33:25.144005: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2022-08-23 02:34:06.728152: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 02:34:06.994378: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2022-08-23 02:34:43.749006: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 02:34:44.005988: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2022-08-23 02:35:23.193065: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 02:35:23.451675: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2022-08-23 09:41:29.967570: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 09:41:31.649475: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 09:42:10.795888: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 09:42:11.252021: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Oracle triggered exit


2022-08-23 09:42:14.556331: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 09:42:16.238194: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 09:42:54.923890: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 09:42:55.379411: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Oracle triggered exit


2022-08-23 09:42:58.675629: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 09:43:00.369980: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 09:43:38.736057: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 09:43:39.200674: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
